# Clustering Experiment 
#### By: Charbel Marche

We decided to individually tackle the problem using 1 method, and by the time we are all done we will be able to merge our techniques and select the optimal techinque. Currently we are determining only the indiviudal digits, but we need to recognize these as coherent numbers and be able to assign entries to numbers.

### Register Images to Start

To start, we need to register images using the `utilities/conversion/apply_homography_to_labels.ipynb` notebook. This should be run before running this notebook. This notebook is built on the assumption that the `data/registered_images` directory has been created and populated. Additionally it assumes that the `data/yolo_data.json` file is created. Both of these are created in the referenced notebook. 

#### Install Packages

These are the necessary packages to run the functions and scripts below.

In [21]:
import os
import json
import random
from pathlib import Path
from typing import List, Tuple, Dict

import cv2
import numpy as np
from PIL import Image, ImageDraw
from sklearn.cluster import KMeans

#### Start By Loading YOLO Data

To start I want to bring in the YOLO formatted data for each sheet and I can additionally load the respective images. As mentioned above you must have ran the `utilities/conversion/apply_homography_to_labels.ipynb` notebook to generate this YOLO data.

In [22]:
# Load yolo_data.json
PATH_TO_YOLO_DATA = '../../data/yolo_data.json'
PATH_TO_REGISTERED_IMAGES = '../../data/registered_images'
UNIFIED_IMAGE_PATH = '../../data/unified_intraoperative_preoperative_flowsheet_v1_1_front.png'
with open(PATH_TO_YOLO_DATA) as json_file:
    yolo_data = json.load(json_file)

# See how many intraoperative images are registered
print(f"Found {len(yolo_data)} sheets in yolo_data.json")

Found 19 sheets in yolo_data.json


Now let's select relevant bounding boxes from the blood pressure and HR zone. 

Start by defining functions to convert YOLO bounding box format to pixels (to see if the bounding box is within region of interest). Then create a function that allows you to select ROI and returns a list of bounding boxes within this ROI.

In [23]:
def YOLO_to_pixels(x_center, y_center, width, height, image_width, image_height):
    """
    Convert YOLO bounding box format to pixel coordinates

    Args:
        x_center: float, x center of the bounding box
        y_center: float, y center of the bounding box
        width: float, width of the bounding box
        height: float, height of the bounding box
        image_width: int, width of the image
        image_height: int, height of the image

    Returns:
        A single tuple with the following values:
            x_min: int, minimum x coordinate of the bounding box in pixels
            y_min: int, minimum y coordinate of the bounding box in pixels
            x_max: int, maximum x coordinate of the bounding box in pixels
            y_max: int, maximum y coordinate of the bounding box in pixels
    """
    x_min = int((float(x_center) * image_width) - (width * image_width) / 2)
    y_min = int((float(y_center) * image_height) - (height * image_height) / 2)
    x_max = int((float(x_center) * image_width) + (width * image_width) / 2)
    y_max = int((float(y_center) * image_height) + (height * image_height) / 2)
    return x_min, y_min, x_max, y_max

# Function to determine whether a point is above or below the diagonal line
def is_point_in_above(x_center, y_center, m, b):
    """
    Determine if a point is above or below the diagonal line y = mx + b.
    For our purposes we use it to check if a bounding box is in the top-right region -- meaning time labels.

    Args:
        x_center: float, x coordinate of the point
        y_center: float, y coordinate of the point
        m: float, slope of the diagonal line
        b: float, intercept of the diagonal line
    
    Returns:
        bool, True if the point is above the line, False otherwise
    """
    # Calculate the y value on the line for the given x_center
    y_line = m * x_center + b
    return y_center > y_line



def select_relevant_bounding_boxes(sheet_data: List[str], path_to_image: Path, show_images: bool = False) -> Tuple[List[str], List[str]]:
    """
    Given sheet data for bounding boxes in YOLO format, display the image and allow the user to select a region of interest (ROI).
    Identify bounding boxes that are within the selected region and draw rectangles around them. 
    Return the bounding boxes that are within the selected region split into two lists: time labels and numerical values.

    Args:
        sheet_data: List of bounding boxes in YOLO format.
        path_to_image: Path to the image file.

    Returns:
        Tuple of Lists of string representations of bounding boxes that are within the selected region, in YOLO format.
        The first list contains bounding boxes in the top-right region -- representing time labels.
        The second list contains bounding boxes in the bottom-left region -- representing numerical values for mmHg and bpm.
            (bounding_boxes_time, bounding_boxes_numbers)
    """
    # Load the image
    image = cv2.imread(path_to_image)

    # Display the image and allow the user to select a ROI
    resized_image = cv2.resize(image, (800, 600))
    roi = cv2.selectROI("Select Region of Interest", resized_image)
    print(f"ROI selected: {roi}")

    # Unpack ROI
    x, y, w, h = roi
    print(f"Selected region: x={x}, y={y}, w={w}, h={h}")

    # Calculate the coordinates of the top-left and bottom-right corners of the selected region
    x_top_left = x
    y_top_left = y
    x_bottom_right = x + w
    y_bottom_right = y + h

    # Draw the diagonal line of the selected region from top-left to bottom-right
    cv2.line(resized_image, (x_top_left, y_top_left), (x_bottom_right, y_bottom_right), (0, 255, 0), 1)
    # Calculate the slope (m) and intercept (b) of the diagonal line.
    # This will allow us to determine if a bounding box is in the top-right region or bottom-left region
    # Top-right region is where time labels are located
    # Bottom-left region is where numerical values for mmHg and bpm are located
    m = (y_bottom_right - y_top_left) / (x_bottom_right - x_top_left)
    b = y_top_left - m * x_top_left

    # List of bounding boxes in the top-right and bottom-left regions
    bounding_boxes_time = []
    bounding_boxes_numbers = []

    # Process the bounding boxes
    for bounding_box in sheet_data:
        # Bounding boxes are in YOLO format; convert them to pixels
        identifier, x_center, y_center, bb_width, bb_height = bounding_box.split(' ') # Identifier is the value in the bounding box, we don't need that here
        x_min, y_min, x_max, y_max  = YOLO_to_pixels(float(x_center), float(y_center), float(bb_width), float(bb_height), 800, 600)
        
        # Check if the bounding box is within the selected region
        if x_min >= x and y_min >= y and x_max <= x + w and y_max <= y + h:
            # Calculate the center of the bounding box
            x_center_bb = (x_min + x_max) / 2
            y_center_bb = (y_min + y_max) / 2
            
            # If we want to generalize this function we can add the option to disregard the diagonal line

            # Determine if the bounding box center is in the top-right region
            if is_point_in_above(x_center_bb, y_center_bb, m, b):
                # Bounding box is in the top-right region
                cv2.rectangle(resized_image, (x_min, y_min), (x_max, y_max), (255, 255, 0), 1)
                bounding_boxes_numbers.append(bounding_box)
            else:
                # Bounding box is in the bottom-left region
                cv2.rectangle(resized_image, (x_min, y_min), (x_max, y_max), (255, 0, 255), 1)
                bounding_boxes_time.append(bounding_box)

    # Close all OpenCV windows, always do this or it will annoyingly not go away
    # You can also manually quit out with ESC key.
    cv2.destroyAllWindows()

    # If we are showing the images, display the image with the selected region and bounding boxes
    # Bounding boxes in the top-right region (time) are in one color while those in the bottom left (numerical) are in another
    if show_images:
        # Display the image with the selected region and bounding boxes
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
        resized_image = Image.fromarray(resized_image)
        resized_image.show()

    # Return a tuple of bounding boxes in the top-right and bottom-left regions
    return (bounding_boxes_time, bounding_boxes_numbers)


Create a function for K-means clustering

In [24]:
def cluster_kmeans(bounding_boxes: List[str], number_of_clusters: int) -> List[int]:
    """
    Cluster bounding boxes using K-Means clustering algorithm.

    Args:
        bounding_boxes: List of bounding boxes in YOLO format.
        number_of_clusters: Number of clusters to use in K-Means clustering.

    Returns:
        List of cluster labels.
    """
    # Convert to a NumPy array (using only x_center and y_center)
    data = np.array([[float(box.split(' ')[1]), float(box.split(' ')[2])] for box in bounding_boxes])

    # Apply K-Means
    kmeans = KMeans(n_clusters=number_of_clusters, init='k-means++', n_init=10, max_iter=300, tol=1e-8, random_state=42)
    kmeans.fit(data)

    # Get cluster labels
    labels = kmeans.predict(data)

    return labels



Function to create a result dictionary that we can save as a JSON file to analyze performance.

In [25]:
def create_result_dictionary(labels: List[str], bounding_boxes: List[str]) -> Dict[int, int]:
    """
    Create a dictionary with cluster labels as keys and lists of bounding boxes as values.

    Args:
        labels: List of cluster labels.
        bounding_boxes: List of bounding boxes in YOLO format.

    Returns:
        Dictionary with cluster labels as keys and bounding box values as values.
    """
    # Create a dictionary to store labelled elements
    label_dict = {}

    # Iterate over both lists
    for label, element in zip(labels, bounding_boxes):
        label = int(label)
        element = str(element)
        if label not in label_dict:
            # Create a new list for this label if it doesn't exist
            label_dict[label] = []
        # Append the element to the corresponding label list
        label_dict[label].append(element)

    # Sort the lists in the dictionary by x_center
    for key in label_dict:
        label_dict[key] = sorted(label_dict[key], key=lambda x: float(x.split(' ')[1]))
        label_dict[key] = [element.split(' ')[0] for element in label_dict[key]]
        # Turn list of strings into a string
        label_dict[key] = int(''.join(label_dict[key]))
    
    return label_dict

Now lets use these functions to get the relevant bounding boxes for clustering.

In [28]:

# Iterate over all images and their bounding boxes
for sheet, bounding_boxes in yolo_data.items():
    print(f"Sheet: {sheet}")
    full_image_path = os.path.join(PATH_TO_REGISTERED_IMAGES, sheet)
    print(f"Full image path: {full_image_path}")

    # Call the analyze_sheet function with data from the loop
    time_bounding_boxes, number_bounding_boxes = select_relevant_bounding_boxes(bounding_boxes, full_image_path)

    # Now we need to cluster the bounding boxes that pertain to the same multi-digit number
    time_labels = cluster_kmeans(time_bounding_boxes, 42)
    number_labels = cluster_kmeans(number_bounding_boxes, 20)

    # Create an image object
    image: Image = Image.open(full_image_path)
    image_width, image_height = image.size

    label_color_map = {}
    for i, label in enumerate(time_labels):

        # Get the bounding box
        bounding_box = time_bounding_boxes[i]
        value, x_center, y_center, width, height = bounding_box.split(' ')
        x_min, y_min, x_max, y_max = YOLO_to_pixels(float(x_center), float(y_center), float(width), float(height), image_width, image_height)

        # Draw bounding boxes on the image
        generate_color = lambda: "#%06x" % random.randint(0, 0xFFFFFF)


        # If the label is not in the color map, generate a new color
        if label not in label_color_map:
            label_color_map[label] = generate_color()

        # Open the image
        draw = ImageDraw.Draw(image)

        box = [
            x_min,
            y_min,
            x_max,
            y_max,
        ]
        draw.rectangle(box, outline=label_color_map[label], width=3)

    # Save the image with the bounding boxes to the kmeans_clustered_images folder
    image.save(f'../../data/kmeans_clustered_images/time/{sheet}')

    # Save the clustered bounding boxes to a JSON file
    with open(f'../../data/kmeans_clustered_images/results/time/{sheet}.json', 'w') as f:
        json.dump(create_result_dictionary(time_labels, time_bounding_boxes), f)

    # Create an image object
    image: Image = Image.open(full_image_path)
    image_width, image_height = image.size

    label_color_map = {}
    for i, label in enumerate(number_labels):

        # Get the bounding box
        bounding_box = number_bounding_boxes[i]
        value, x_center, y_center, width, height = bounding_box.split(' ')
        x_min, y_min, x_max, y_max = YOLO_to_pixels(float(x_center), float(y_center), float(width), float(height), image_width, image_height)

        # Draw bounding boxes on the image
        generate_color = lambda: "#%06x" % random.randint(0, 0xFFFFFF)


        # If the label is not in the color map, generate a new color
        if label not in label_color_map:
            label_color_map[label] = generate_color()

        # Open the image
        draw = ImageDraw.Draw(image)

        box = [
            x_min,
            y_min,
            x_max,
            y_max,
        ]
        draw.rectangle(box, outline=label_color_map[label], width=3)

    # Save the image with the bounding boxes to the kmeans_clustered_images folder
    image.save(f'../../data/kmeans_clustered_images/number/{sheet}')

    # Save the clustered bounding boxes to a JSON file
    with open(f'../../data/kmeans_clustered_images/results/number/{sheet}.json', 'w') as f:
        json.dump(create_result_dictionary(number_labels, number_bounding_boxes), f)

Sheet: RC_0001_intraoperative.JPG
Full image path: ../../data/registered_images\RC_0001_intraoperative.JPG
ROI selected: (102, 221, 634, 203)
Selected region: x=102, y=221, w=634, h=203
Sheet: RC_0002_intraoperative.JPG
Full image path: ../../data/registered_images\RC_0002_intraoperative.JPG
ROI selected: (102, 220, 635, 204)
Selected region: x=102, y=220, w=635, h=204
Sheet: RC_0003_intraoperative.JPG
Full image path: ../../data/registered_images\RC_0003_intraoperative.JPG
ROI selected: (102, 220, 634, 204)
Selected region: x=102, y=220, w=634, h=204
Sheet: RC_0004_intraoperative.JPG
Full image path: ../../data/registered_images\RC_0004_intraoperative.JPG
ROI selected: (104, 222, 631, 201)
Selected region: x=104, y=222, w=631, h=201
Sheet: RC_0005_intraoperative.JPG
Full image path: ../../data/registered_images\RC_0005_intraoperative.JPG
ROI selected: (103, 221, 631, 203)
Selected region: x=103, y=221, w=631, h=203
Sheet: RC_0006_intraoperative.JPG
Full image path: ../../data/register

#### Create Cluster Mapping
Contains a cluster number that maps list of numbers that belong to that cluster *in-order*

This will allow us to easily impute meanings of the labels since we have them split by time and number (mmHg and HR)

In [27]:
# Since this work is done above, we can simply read in from the JSON files created in the previous step and work from there.

# Paths to the JSON files
PATH_TO_KMEANS_RESULTS = '../../data/kmeans_clustered_images/results'
TIME_JSON = os.path.join(PATH_TO_KMEANS_RESULTS, 'time')
NUMBER_JSON = os.path.join(PATH_TO_KMEANS_RESULTS, 'number')

final_time_clusters = {}
final_number_clusters = {}

# Iterate over all images and their bounding boxes
for sheet, bounding_boxes in yolo_data.items():
    # Load JSON
    with open(os.path.join(TIME_JSON, f'{sheet}.json')) as f:
        time_clusters = json.load(f)
    
    # Each cluster contains the number (integer) that the cluster represents
    # We know what integers should be represented in the time labels, lets check that they are all there.
    # Keep track of any false positives (new clusters that don't exist) or negatives (missing clusters)
    

    


IndexError: list index out of range

#### Analyze accuracy

Below we use assumptions on what we know the labels should represent in both the time and number groups. We check that these values are present within clusters.

In [7]:
def print_classes_and_values(classes, values):
  """Prints the class number followed by all the indices and values belonging to that class.

  Args:
    classes: A list of class labels for each value.
    values: A list of values.
  """

  # Create a dictionary to store indices and values by class
  class_data = {}
  for i, class_label in enumerate(classes):
    if class_label not in class_data:
      class_data[class_label] = []
    class_data[class_label].append((i, values[i]))

  # Print the class number followed by the indices and values
  for class_label, data in class_data.items():
    print(f"Class {class_label}:")
    for index, value in data:
      print(f"  Index {index}: {value}")


relevant_numbers = [box.split(" ")[0] for box in relevant_bounding_boxes]
print_classes_and_values(labels, relevant_numbers)


NameError: name 'relevant_bounding_boxes' is not defined